In [123]:
import configparser
import requests
from requests import Session
from psycopg2 import connect
from psycopg2.extras import execute_values
from time import sleep
CONFIG = configparser.ConfigParser()
CONFIG.read(r'C:\Users\nchan6\Documents\db.cfg')
dbset = CONFIG['DBSETTINGS']
con = connect(**dbset)

In [214]:
def get_url(mapserver, id):
    "This prints the base_url for the request"
    url = "https://insideto-gis.toronto.ca/arcgis/rest/services/{}/MapServer/{}/query".format(mapserver, id)
    return(url)

In [233]:
def check_status(max_number, record_max):
        try:
            r = requests.get(base_url, params = query, verify = False)
            print('trying')
        except requests.exceptions.SSLError:
            sleep(30)
            status = 'do it again'
        else:
            status = 'its k'
        return status

In [271]:
def get_layer1(mapserver, id, output_table):
    base_url = get_url({}, {}).format(mapserver, id)
    query = {"where":"1=1",
             "outFields": "*",
             "outSR": '4326',         
             "returnGeometry": "true",
             "returnTrueCurves": "false",
             "returnIdsOnly": "false",
             "returnCountOnly": "false",
             "returnZ": "false",
             "returnM": "false",
             "orderByFields": "OBJECTID", 
             "returnDistinctValues": "false",
             "returnExtentsOnly": "false",
             "f":"json"}
    # First get request to get maxnumber and append first request
    r = requests.get(base_url, params = query, verify = False)
    return_json = r.json()
    features = return_json['features']
    record_max=(len(features))
    max_number = record_max
    rows = []
    # We first append the first set of features
    for feature in features:
        row = (feature['attributes']['CENTRELINE_ID'], 
                feature['attributes']['TRANS_ID_CREATE'],
                feature['attributes']['OBJECTID'],
                feature['attributes']['MI_PRINX'],
                feature['attributes']['REC_ID'],
                feature['attributes']['TYPE_CODE'],
                feature['attributes']['TYPE_DESC'],
                feature['attributes']['STATUS_CODE'],
                feature['attributes']['STATUS_DESC'],
                'SRID=4326;LineString('+','.join(' '.join(str(x) for x in tup) for tup in feature['geometry']['paths'][0]) +')')
            
        rows.append(row)
            
        # send to database    
    sql='INSERT INTO {} (geo_id, trans_id_create, object_id, mi_prinx, rec_id, type_code, type_desc, status_code, status_desc, geometry) VALUES %s'.format(output_table)
        
    with con:
        with con.cursor() as cur:
            execute_values(cur, sql, rows) 
    print('inserted first set')    
    if return_json.get('exceededTransferLimit', False) == True:
        print('going with while loop')
        # Then loop the rest of the features
        while return_json.get('exceededTransferLimit', False) == True or check_status == 'do it again':
                print('Current loop:', max_number, record_max)
                query = {"where": "1=1", 
                         "outFields": "*",
                         "outSR": '4326',         
                         "returnGeometry": "true",
                         "returnTrueCurves": "false",
                         "returnIdsOnly": "false",
                         "returnCountOnly": "false",
                         "returnZ": "false",
                         "returnM": "false",
                         "returnDistinctValues": "false",
                         "resultOffset": "{}".format(max_number),
                         "resultRecordCount": "{}".format(record_max),
                         "orderByFields": "OBJECTID", 
                         "returnExtentsOnly": "false",
                         "f":"json"}

                check_status1 = check_status(max_number, record_max)
                
                if check_status1 == 'its k':
                    r = requests.get(base_url, params = query, verify = False)
                    return_json = r.json()
                    features = return_json['features']
                    
                    print('length:', len(features))
                    rows = []
                    # We first append the first set of features
                    for feature in features:
                        row = (feature['attributes']['CENTRELINE_ID'], 
                               feature['attributes']['TRANS_ID_CREATE'],
                               feature['attributes']['OBJECTID'],
                               feature['attributes']['MI_PRINX'],
                               feature['attributes']['REC_ID'],
                               feature['attributes']['TYPE_CODE'],
                               feature['attributes']['TYPE_DESC'],
                               feature['attributes']['STATUS_CODE'],
                               feature['attributes']['STATUS_DESC'],
                               'SRID=4326;LineString('+','.join(' '.join(str(x) for x in tup) for tup in feature['geometry']['paths'][0]) +')')
                        rows.append(row)
                    # send to database    
                    sql='INSERT INTO {} (geo_id, trans_id_create, object_id, mi_prinx, rec_id, type_code, type_desc, status_code, status_desc, geometry) VALUES %s'.format(output_table)

                    with con:
                        with con.cursor() as cur:
                            execute_values(cur, sql, rows)       
                    print('inserted')
                    max_number = max_number + record_max
                    print('Next max_number: ', max_number)
                    
                else:
                    print('what happened')
        else:
            print('dont need to loop')

In [269]:
get_layer1('cot_geospatial2', 2, 'gcc_table')

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


inserted first set
going with while loop
Current loop: 1000 1000


C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


trying
{'where': '1=1', 'outFields': '*', 'outSR': '4326', 'returnGeometry': 'true', 'returnTrueCurves': 'false', 'returnIdsOnly': 'false', 'returnCountOnly': 'false', 'returnZ': 'false', 'returnM': 'false', 'returnDistinctValues': 'false', 'resultOffset': '1000', 'resultRecordCount': '1000', 'orderByFields': 'OBJECTID', 'returnExtentsOnly': 'false', 'f': 'json'}
length: 1000
17429
17430
17442
17447
17448
17449
17454
17515
17561
17569
17579
17623
17624
17662
17669
17670
17715
17721
17737
17738
17739
17741
17747
17751
17755
17771
17778
17782
17785
17786
17796
17802
17828
17833
17834
17839
17840
17842
17857
17903
17906
17909
17910
17932
17933
17934
17936
17937
17979
18068
18069
18079
18080
18083
18100
18109
18111
18116
18117
18121
18123
18129
18134
18138
18144
18226
18244
18247
18248
18260
18276
18284
18285
18297
18338
18340
18356
18358
18381
18388
18389
18400
18425
18440
18453
18461
18475
18533
18534
18539
18577
18581
18582
18583
18584
18586
18587
18607
18616
18617
18618
18620
18639
186

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


...

trying
{'where': '1=1', 'outFields': '*', 'outSR': '4326', 'returnGeometry': 'true', 'returnTrueCurves': 'false', 'returnIdsOnly': 'false', 'returnCountOnly': 'false', 'returnZ': 'false', 'returnM': 'false', 'returnDistinctValues': 'false', 'resultOffset': '6000', 'resultRecordCount': '1000', 'orderByFields': 'OBJECTID', 'returnExtentsOnly': 'false', 'f': 'json'}
length: 1000
206104
206105
206121
206133
206135
206147
206152
206158
206161
206162
206163
206223
206237
206238
206240
206241
206257
206265
206287
206293
206296
206303
206304
206306
206307
206310
206311
206312
206315
206445
206446
206454
206474
206479
206559
206577
206578
206579
206591
206621
206635
206636
206640
206641
206646
206647
206648
206649
206706
206716
206721
206724
206820
206825
206826
206829
206830
206838
206840
206872
206895
206896
206897
206898
206978
207029
207030
207086
207087
207166
207168
207290
207295
207338
207339
207346
207347
207421
207429
207430
207431
207585
207586
207598
207603
207607
207609
207629
20763

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


trying
{'where': '1=1', 'outFields': '*', 'outSR': '4326', 'returnGeometry': 'true', 'returnTrueCurves': 'false', 'returnIdsOnly': 'false', 'returnCountOnly': 'false', 'returnZ': 'false', 'returnM': 'false', 'returnDistinctValues': 'false', 'resultOffset': '7000', 'resultRecordCount': '1000', 'orderByFields': 'OBJECTID', 'returnExtentsOnly': 'false', 'f': 'json'}


C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


length: 49
220186
220189
220190
220191
220192
220201
220202
220212
220220
220221
220222
220223
220318
220321
220322
220323
220324
220400
220402
220403
220515
220517
220518
220538
220554
220600
220612
220634
220651
220652
220658
220699
220720
220734
220735
220736
220737
220771
220772
220786
220789
220790
220795
220796
220797
220799
220812
220814
220821
inserted
Next max_number:  8000
dont need to loop


In [98]:
recordmax = 1000
max_number = recordmax
print(max_number)
while max_number<7000 :
    query = {"where": "1+1", 
                 "outFields": "*",
                 "outSR": '4326',         
                 "returnGeometry": "true",
                 "returnTrueCurves": "false",
                 "returnIdsOnly": "false",
                 "returnCountOnly": "false",
                 "returnZ": "false",
                 "returnM": "false",
                 "returnDistinctValues": "false",
                 "returnExtentsOnly": "false",
                 "resultOffset": "{}".format(max_number),
                 "resultRecordCount": "{}".format(recordmax),
                 "f":"json"}
    print("max_number:", max_number)
    print("max_record:",recordmax)
    max_number = max_number + recordmax
    try: 
        max_number == 7000
    except : 
        print('not yet')
    else: 
        print('yay')

1000
max_number: 1000
max_record: 1000
yay
max_number: 2000
max_record: 1000
yay
max_number: 3000
max_record: 1000
yay
max_number: 4000
max_record: 1000
yay
max_number: 5000
max_record: 1000
yay
max_number: 6000
max_record: 1000
yay


In [110]:
def get_layer(mapserver, id, output_table):
    
    """
    This function calls to the GCCview rest API and inserts the outputs to the output table in the postgres database.
    
    Parameters
    ----------
    mapserver : string
        The name of the mapserver that host the desire layer
        
    id : int
        The id of desire layer
        
    output_table: string
        Name of table that returned row will be inserted into
    """
    base_url = get_url({},{}).format(mapserver, id)
    print('Getting object_id...')
    object_id = get_objectid('OBJECTID>=1')
    print('object_id returned')
    
    # first call to api, check if the object_ids are too long
    query = {"where":"OBJECTID in {}".format(tuple(object_id)),
    "text": "",
    "objectIds":"", 
    "outFields": "*",
    "outSR": '4326',         
    "returnGeometry": "true",
    "returnTrueCurves": "false",
    "returnIdsOnly": "false",
    "returnCountOnly": "false",
    "returnZ": "false",
    "returnM": "false",
    "returnDistinctValues": "false",
    "returnExtentsOnly": "false",
    "f": "json"}

    
    r = requests.get(base_url, params = query, verify=False)
    
    # if the url is too long    
    if str(r) == '<Response [414]>':
        print('url too long, proceed with for loop')
        loop_id = []
        for i in range(0, len(object_id), 49):    
            if i == 0:
                stuff = (object_id[0:i+50])
                loop_id.append(stuff)
            else:
                stuff = (object_id[i+1:i+50])
                loop_id.append(stuff)
       
        # loop though 50 of objectid each time
        for i in loop_id:
            query = {"where":"OBJECTID in {}".format(tuple(i)),
                    "text": "",
                    "objectIds":"", 
                    "outFields": "*",
                    "outSR": '4326',         
                    "returnGeometry": "true",
                    "returnTrueCurves": "false",
                    "returnIdsOnly": "false",
                    "returnCountOnly": "false",
                    "returnZ": "false",
                    "returnM": "false",
                    "returnDistinctValues": "false",
                    "returnExtentsOnly": "false",
                    "f": "json"}
            r = requests.get(base_url, params = query, verify=False)
            return_json = r.json()
            features = return_json['features']
            rows = []
            for feature in features:
                row = (feature['attributes']['CENTRELINE_ID'], 
                       feature['attributes']['TRANS_ID_CREATE'],
                       feature['attributes']['OBJECTID'],
                       feature['attributes']['MI_PRINX'],
                       feature['attributes']['REC_ID'],
                       feature['attributes']['TYPE_CODE'],
                       feature['attributes']['TYPE_DESC'],
                       feature['attributes']['STATUS_CODE'],
                       feature['attributes']['STATUS_DESC'],
                       'SRID=4326;LineString('+','.join(' '.join(str(x) for x in tup) for tup in feature['geometry']['paths'][0]) +')')

                rows.append(row)

            # send to the database
            sql='INSERT INTO {} (geo_id, trans_id_create, object_id, mi_prinx, rec_id, type_code, type_desc, status_code, status_desc, geometry) VALUES %s'.format(output_table)

            with con:
                with con.cursor() as cur:
                    execute_values(cur, sql, rows) 

    # if url is not too long 
    else: 
        return_json = r.json()
        features = return_json['features']
        rows = []
        for feature in features:
            row = (feature['attributes']['CENTRELINE_ID'], 
                   feature['attributes']['TRANS_ID_CREATE'],
                   feature['attributes']['OBJECTID'],
                   feature['attributes']['MI_PRINX'],
                   feature['attributes']['REC_ID'],
                   feature['attributes']['TYPE_CODE'],
                   feature['attributes']['TYPE_DESC'],
                   feature['attributes']['STATUS_CODE'],
                   feature['attributes']['STATUS_DESC'],
                   'SRID=4326;LineString('+','.join(' '.join(str(x) for x in tup) for tup in feature['geometry']['paths'][0]) +')')
            
            rows.append(row)
            
        # send to database    
        sql='INSERT INTO {} (geo_id, trans_id_create, object_id, mi_prinx, rec_id, type_code, type_desc, status_code, status_desc, geometry) VALUES %s'.format(output_table)
        
        with con:
            with con.cursor() as cur:
                execute_values(cur, sql, rows) 


In [26]:
return_json

NameError: name 'return_json' is not defined

In [241]:
get_layer('cot_geospatial2', 2, 'natalie.gcc_table')

Getting object_id...
object_id returned


C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


url too long, proceed with for loop


C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTT